In [67]:
!pip install snscrape
import pandas as pd
import snscrape.modules.twitter as sntwitter
import datetime
from datetime import timedelta
from tqdm.notebook import tqdm_notebook
from google.colab import files

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [70]:
def search(text, username, since, until, retweet, replies, language):
    global filename
    q = text
    if username!='':
        q += f" from:{username}"    
    if until=='':
        until = datetime.datetime.strftime(datetime.date.today(), '%Y-%m-%d')
    q += f" until:{until}"
    if since=='':
        since = datetime.datetime.strftime(datetime.datetime.strptime(until, '%Y-%m-%d') - 
                                           datetime.timedelta(days=7), '%Y-%m-%d')
    q += f" since:{since}"
    if retweet == 'y':
        q += f" exclude:retweets"
    if replies == 'y':
        q += f" exclude:replies"
    if username!='' and text!='':
        filename = f"{since}_{until}_{username}_{text}.csv"
    elif username!="":
        filename = f"{since}_{until}_{username}.csv"
    else:
        filename = f"{since}_{until}_{text}.csv"
    if language != '':
      q += f" lang:{language}"
    print(filename)
    return q

In [71]:
text = 'bitcoin'
username = ''
since = '2013-04-28'
until = '2023-04-17'
count = 100
retweet = 'y'
replies = 'y'
language = 'en'

# Creating list to append tweet data 
tweets_list1 = []

start_date = datetime.datetime.strptime(since, '%Y-%m-%d').replace(
    hour=0, minute=0, second=0, microsecond=0)  # start date
end_date = datetime.datetime.strptime(until, '%Y-%m-%d').replace(
    hour=0, minute=0, second=0, microsecond=0)  # end date

if count == -1:
    for i,tweet in enumerate(tqdm_notebook(
        sntwitter.TwitterSearchScraper(q).get_items())):
        tweets_list1.append([tweet.date, tweet.rawContent,tweet.lang,
                             tweet.replyCount,tweet.retweetCount,
                             tweet.likeCount,tweet.quoteCount])
else:
  for n in range((end_date - start_date).days + 1):
    date = start_date + timedelta(n)
    date_plus_one_day = date + timedelta(days=1)
    q = search(text,username,date.strftime('%Y-%m-%d'),
               date_plus_one_day.strftime('%Y-%m-%d'), retweet, replies,
               language)
    with tqdm_notebook(total=count) as pbar:
        for i,tweet in enumerate(sntwitter.TwitterSearchScraper(q).get_items()): #declare a username 
            if i>=count: #number of tweets you want to scrape
                break
            tweets_list1.append([tweet.date, tweet.rawContent,tweet.lang,
                                 tweet.replyCount,tweet.retweetCount,
                                 tweet.likeCount,tweet.quoteCount])
            pbar.update(1)
# Creating a dataframe from the tweets list above 
tweets_df1 = pd.DataFrame(tweets_list1, columns=['DateTime', 'Text', 'Language',
                                'ReplyCount','RetweetCount','LikeCount','QuoteCount'])
tweets_df1

2013-04-30_2013-05-01_bitcoin.csv
bitcoin until:2013-05-01 since:2013-04-30 exclude:retweets exclude:replies lang:en


  0%|          | 0/100 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

In [63]:
tweets_df1.sort_values(by='DateTime', ascending=False)

,DateTime,Text,Language,ReplyCount,RetweetCount,LikeCount,QuoteCount
364100,2023-04-17 19:37:57+00:00,Version 1.5.6 has been released which includes...,en,1,0,0,0
364101,2023-04-17 19:37:55+00:00,#Bitcoin 1Mi 😎🍸 https://t.co/Bi7GjL8xzC,und,0,0,0,0
364102,2023-04-17 19:37:47+00:00,Enfia ad de bitcoin no cy,es,0,0,0,0
364103,2023-04-17 19:37:44+00:00,Valentía para mantener nuestros principios y o...,es,0,0,0,0
364104,2023-04-17 19:37:39+00:00,#bitcoin is revolutionary 🤾🏿‍♂️,en,0,0,0,0
...,...,...,...,...,...,...,...
95,2013-04-28 23:20:15+00:00,Last trade: 103.99€ -- Avg. 24h: 103.61€ | 7d:...,en,0,0,0,0
96,2013-04-28 23:18:55+00:00,The bitcoin gem is worth 0.38 bitcoins http://...,en,0,0,0,0
97,2013-04-28 23:18:45+00:00,"Bitcoin - $35 in Feb, $80 at the end of March,...",en,0,0,1,0
98,2013-04-28 23:18:20+00:00,ObjectiveBitcoin sentiment analysis Indicator:...,en,0,0,0,0


In [64]:
# Make the tweet data from DataFrame to .csv file
tweets_df1.to_csv('bitcoin_tweets.csv')

In [66]:
# Download the data to the computer
files.download("bitcoin_tweets.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>